In [69]:
import sys
!{sys.executable} -m pip install rdflib yatter morph-kgc

In [2]:
from os import getcwd
from os.path import join, dirname
folder = dirname(getcwd())

In [46]:
from rdflib import Graph
# Load RDF data from file
g = Graph()
rdf_file = join(folder, "rdf/digital-competence-framework-skos-ap-eu.rdf")  # Replace with the actual path to your RDF file
g.parse(rdf_file, format="xml")     # Specify the appropriate format if not XML

# SPARQL query
sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    
    SELECT DISTINCT ?label ?id
    WHERE {
      ?s skos:prefLabel ?label ;
          dc:identifier ?id .
      FILTER (!CONTAINS(?label, 'Level'))
    }
    ORDER BY ?label
"""

# Execute the SPARQL query
results = g.query(sparql_query)

# Print the query results
for row in results:
    label = str(row.label)[:3].strip()
    if len(label) == 2:
        label = label[:1]
    print(f"{label},{row.id}")

1,8b23nt41l4
1.1,9i6ta81748
1.2,3h9eab1489
1.3,4s23kdq383
2,2k3209g1wc
2.1,8ys96p08p4
2.2,9a5d39t62b
2.3,1dz4n8r554
2.4,y2s2861s9p
2.5,99vvo91542
2.6,7d4t7002v7
3,4b465z8p2e
3.1,u69o196gu6
3.2,sq0c9q6116
3.3,2mi55kl065
3.4,0690r7o8r2
4,1fz48677f4
4.1,aa985fr035
4.2,1b043jw49b
4.3,mg54my1580
4.4,o9e8s72z21
5,86xdxt8620
5.1,13z458t54u
5.2,00b51s9rc5
5.3,3bgx2a6278
5.4,860966ekgo


In [47]:
import yatter
from ruamel.yaml import YAML

# YARRRML to RML
yaml = YAML(typ='safe', pure=True)
mappings_file = join(folder, "mappings/corso-kg-yarrrml.yml")
rml_content = yatter.translate(yaml.load(open(mappings_file)))

# Print generated RML triples
# print(rml_content)

# Save RML to file
rml_file = join(folder, "mappings/corso-kg-rml.ttl")
with open(rml_file, "w") as rml_output:
    rml_output.write(rml_content)

2023-07-18 13:17:35,980 | INFO: Translating YARRRML mapping to [R2]RML
2023-07-18 13:17:35,982 | INFO: RML content is created!
2023-07-18 13:17:35,988 | INFO: Mapping has been syntactically validated.
2023-07-18 13:17:35,989 | INFO: Translation has finished successfully.


In [50]:
import morph_kgc

config = """
            [CONFIGURATION]
            output_file: ./../rdf/knowledge-graph.ttl
            output_format: N-TRIPLES
            
            [Competences]
            mappings: ./../mappings/corso-kg-rml.ttl
         """
# Generate the triples and load them to an RDFLib graph
g_rdflib = morph_kgc.materialize(config)
# Save to file
output_file = join(folder, "rdf/knowledge-graph.ttl")
g_rdflib.serialize(destination=output_file, format='turtle')

2023-07-18 13:19:29,823 | DEBUG: CONFIGURATION: {'output_file': './../rdf/knowledge-graph.ttl', 'output_format': 'N-TRIPLES', 'na_values': ',nan', 'safe_percent_encoding': '', 'read_parsed_mappings_path': '', 'write_parsed_mappings_path': '', 'mapping_partitioning': 'PARTIAL-AGGREGATIONS', 'logging_file': '', 'oracle_client_lib_dir': '', 'oracle_client_config_dir': '', 'udfs': '', 'output_dir': '', 'only_printable_chars': 'no', 'infer_sql_datatypes': 'no', 'logging_level': 'INFO', 'number_of_processes': '16'}
2023-07-18 13:19:29,824 | DEBUG: DATA SOURCE `Competences`: {'mappings': './../mappings/corso-kg-rml.ttl'}
2023-07-18 13:19:30,550 | DEBUG: Removed self-join from mapping rule `#TM5`.
2023-07-18 13:19:30,556 | INFO: 11 mapping rules retrieved.
2023-07-18 13:19:30,574 | INFO: Mapping partition with 1 groups generated.
2023-07-18 13:19:30,576 | INFO: Maximum number of rules within mapping group: 11.
2023-07-18 13:19:30,577 | INFO: Mappings processed in 0.747 seconds.
2023-07-18 13:1

<Graph identifier=N320f838c4a2343fd85936af451c7d0da (<class 'rdflib.graph.Graph'>)>

In [51]:
q_res = g_rdflib.query('SELECT (COUNT(*) AS ?num_triples) WHERE { ?s ?p ?o }')

# Print the query results
for row in q_res:
    print(f"Number of triples generated: {row.num_triples}")

Number of triples generated: 1210
